# Inference about effect sizes (IN PROGRESS)
This notebook is going to explore any mistaken research conclusions that may arise from use of the original Rachlin discount function, aand compare that to the Modified Rachlin function.

We will do this in the context of comparing two groups with different discount rates $\kappa$. We will explore a range of true effect sizes (difference in means of 2 groups divided by pooled variance).

Steps are:

1. Define true group means of $\kappa$.
2. Define a range of true effect sizes, varying the variance of $\kappa$ for each group accordingly.
3. Iterate over each true effect size:
    - simulate a dataset of $N$ participants
    - conduct Bayesian inference to estimate effect sizes, doing so for the Rachlin and the Modified Rachlin discount functions.
    - plot the true effect size vs the inferred effect size.
    
**The example below is with differences in group $\kappa$ values, but no group difference in group $s$ values.**

Note that the Bayesian t-test component of this code was inspired by the PyMC3 implementation (here https://github.com/strawlab/best, https://docs.pymc.io/notebooks/BEST.html) of the BEST model by Kruschke (2012).

In [ ]:
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

import numpy as np
import math
import pandas as pd

# import sys
# print("Python version:\n{}\n".format(sys.version))

import pymc3 as pm
print(f'PyMC3 v{pm.__version__}')

%run set_plot_options.py
from discount_functions import rachlin, rachlin_kappa
from adaptive_experiment import make_dataset

In [ ]:
# Initialize random number generator
SEED = 123
np.random.seed(SEED)

# Define sampler options
sample_options = {'tune': 500, 'draws': 1000,
                  'chains': 4, 'cores': 2,
                  'nuts_kwargs': {'target_accept': 0.95},
                  'random_seed': SEED}

## Functions for simulated dataset generation

In [ ]:
def cohend(d1, d2):
    # calculate the size of samples
    n1, n2 = len(d1), len(d2)
    # calculate the variance of the samples
    s1, s2 = np.var(d1, ddof=1), np.var(d2, ddof=1)
    # calculate the pooled standard deviation
    s = math.sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
    # calculate the means of the samples
    u1, u2 = np.mean(d1), np.mean(d2)
    # calculate the effect size
    return (u1 - u2) / s

In [ ]:
def make_participant_level_dataset(logκ_std,
                                   groupA_logκmean=-4.6, 
                                   groupB_logκmean=-2.3,
                                   groupA_logsmean=0, 
                                   groupB_logsmean=0,
                                   logs_std=0.1,
                                   N_per_group=20):
    '''Generate a set of true participant discount function parameter
    values which conforms to the inputs:
    - group means
    - effect size'''
    
    # generate log(κ) values
    a_logκ = np.random.normal(loc=groupA_logκmean, scale=logκ_std, size=N_per_group)
    b_logκ = np.random.normal(loc=groupB_logκmean, scale=logκ_std, size=N_per_group)
    
    # generate log(s) values
    a_logs = np.random.normal(loc=groupA_logsmean, scale=logs_std, size=N_per_group)
    b_logs = np.random.normal(loc=groupB_logsmean, scale=logs_std, size=N_per_group)
    
    dataset = pd.DataFrame()
    dataset['logkappa'] = np.concatenate([a_logκ, b_logκ])
    dataset['logs'] = np.concatenate([a_logs, b_logs])
    dataset['group'] = np.concatenate( (np.zeros(N_per_group), np.ones(N_per_group)))
    # convert to int
    dataset['group'] = np.int_(dataset['group'])
    
    true_logκ_effect_size = cohend(a_logκ, b_logκ)
    
    return dataset, true_logκ_effect_size

In [ ]:
def split_apply_combine(df, func):
    '''Ben's useful split-apply-combine utility function.
    We split by row, iterate over them applying the function, then combine into a
    single dataframe.'''
    output = []
    for i in range(df.shape[0]):
        # split by row
        row = df.iloc[i,:]
        # apply function
        output.append(func(row, i))
        
    # combine
    output = pd.concat(output, ignore_index=True)
    return output


def simulate_one_experiment(row, i):
    
    data_generating_discount_func = rachlin_kappa
    
    # extract info from row (HERE IS WHERE WE CONVERT PARAMETERS FROM LOG TO LINEAR SPACE)
    data_generating_params = np.exp(row.logs), np.exp(row.logkappa)
    
    # apply my function... simulate a dataset
    pdata = make_dataset(data_generating_discount_func, data_generating_params)
    
    # add participant number column
    pdata['id'] = pd.Series(i, index=pdata.index)
    
    # add group column
    group = row.group
    pdata['group'] = np.int_(pd.Series(group, index=pdata.index))
    
    return pdata

In [ ]:
def simulate_entire_experiment(participant_level_dataset):
    '''Simulate an entire experiment based on the provided true participant level
    parameters.'''
    trial_level_dataset = split_apply_combine(participant_level_dataset, simulate_one_experiment)
    return trial_level_dataset

## Functions for Bayesian inference on effect size

In [ ]:
def V(reward, delay, logk, logs):
    '''Calculate the present subjective value of a given prospect'''
    k = pm.math.exp(logk)
    s = pm.math.exp(logs)
    return reward * modified_rachlin(delay, k, s)


def modified_rachlin(delay, k, s):
    ''' This is the MODIFIED Rachlin discount function.'''
    return 1 / (1.0+(k*delay)**s)


def Φ(VA, VB, ϵ=0.01):
    '''Psychometric function which converts the decision variable (VB-VA)
    into a reponse probability. Output corresponds to probability of choosing
    the delayed reward (option B).'''
    return ϵ + (1.0-2.0*ϵ) * (1/(1+pm.math.exp(-1.7*(VB-VA))))

# TODO: Update the Φ function

In [ ]:
def participant_group_lookup(trial_level_dataset):
    # create a group lookup table. 1 entry for each particpant
    temp = np.array([trial_level_dataset['id'].values, trial_level_dataset['group'].values]).T
    temp = np.unique(temp, axis=0)
    group = temp[:,1]
    return group

In [ ]:
def make_modified_rachlin_model(trial_level_dataset):
    '''Returns a PyMC3 model which does inference on discounting parameters from participants
    from two groups. We have hierachical inference at the group level and we are essentially doing
    a Bayesian t-test where we estimate Cohen's D effect size'''
    
    # extract data from DataFrame
    DA = trial_level_dataset.DA.values
    RA = trial_level_dataset.RA.values
    DB = trial_level_dataset.DB.values
    RB = trial_level_dataset.RB.values
    R = trial_level_dataset.R.values
    id = trial_level_dataset.id.values
    
    group = participant_group_lookup(trial_level_dataset)
    
    n_participants = np.max(trial_level_dataset.id) + 1
    n_groups = 2
     
#     # use actual mean and std as parameters for the prior
#     μ_m = y.value.mean()
#     μ_s = y.value.std() * 2

    σ_low = 0
    σ_high = 10
    
    # Start building the model from the top down, starting with Bayesian t-test
    with pm.Model() as model:
        logκ_group_means = pm.Normal('logκ_group_means', mu=-3, sd=0.1, shape=n_groups)
        logκ_group_stds = pm.Uniform('logκ_group_stds', lower=σ_low, upper=σ_high, shape=n_groups)
#         group_logk = pm.Normal('group_logk', mu=group_means, sd=group_stds, shape=n_groups)

        logs_group_means = pm.Normal('logs_group_means', mu=1, sd=0.1, shape=n_groups)
        logs_group_stds = pm.Uniform('logs_group_stds', lower=σ_low, upper=σ_high, shape=n_groups)
            
    # compute measures about the group level estimates of logk
    with model:
        logκ_diff_of_means = pm.Deterministic('logκ difference of means', logκ_group_means[0] - logκ_group_means[1])
        logκ_diff_of_stds = pm.Deterministic('logκ difference of stds', logκ_group_stds[0] - logκ_group_stds[1])
        logκ_effect_size = pm.Deterministic('logκ effect size',
                                       logκ_diff_of_means / np.sqrt((logκ_group_stds[0]**2 + logκ_group_stds[1]**2) / 2))
        
        logs_diff_of_means = pm.Deterministic('logs difference of means', logs_group_means[0] - logs_group_means[1])
        logs_diff_of_stds = pm.Deterministic('logs difference of stds', logs_group_stds[0] - logs_group_stds[1])
        logs_effect_size = pm.Deterministic('logs effect size',
                                       logs_diff_of_means / np.sqrt((logs_group_stds[0]**2 + logs_group_stds[1]**2) / 2))
        
    # Discounting part of the model
    with model:
        # priors over parameters for each participant
        logκ = pm.Normal('logκ', mu=logκ_group_means[group], sd=logκ_group_stds[group], shape=n_participants)
        logs = pm.Normal('logs', mu=logs_group_means[group], sd=logs_group_stds[group], shape=n_participants)
        
        # Choice function: psychometric
        P = pm.Deterministic('P', Φ(V(RA, DA, logκ[id], logs[id]),
                                    V(RB, DB, logκ[id], logs[id])) )
        # Likelihood of observations
        R = pm.Bernoulli('R', p=P, observed=R)

    return model

# High level code here

Define true things about the experiment and groups

In [ ]:
# N = 20
# groupA = {'κ_mean': 0.01, 's_mean':1, 's_std': 0.01}
# groupA = {'κ_mean': 0.001, 's_mean':1, 's_std': 0.01}

# # we iterate of over effect sizes. This is implimented by changing the std on kappa
# group_κstd_list = np.linspace(0.0001, 0.01, 10)

# Test code to do inference for a given effect size

In [ ]:
group_logκstd = 0.01

participant_level_dataset, true_κ_effect_size = make_participant_level_dataset(group_logκstd,
                                   groupA_logκmean=-4.6, 
                                   groupB_logκmean=-2.3,
                                   groupA_logsmean=0, 
                                   groupB_logsmean=0,
                                   logs_std=0.1,
                                   N_per_group=20)

In [ ]:
trial_level_dataset = simulate_entire_experiment(participant_level_dataset)

In [ ]:
model = make_modified_rachlin_model(trial_level_dataset)

In [ ]:
pm.model_to_graphviz(model)

In [ ]:
# DO THE INFERENCE
with model:
    trace = pm.sample(**sample_options)

In [ ]:
pm.energyplot(trace)

In [ ]:
pm.forestplot(trace, var_names=['logκ'], r_hat=True);

In [ ]:
pm.forestplot(trace, var_names=['logs'], r_hat=True);

In [ ]:
pm.forestplot(trace, var_names=['logκ_group_means', 'logκ_group_stds']);

In [ ]:
pm.forestplot(trace, var_names=['logs_group_means', 'logs_group_stds'], r_hat=True);

In [ ]:
pm.plot_posterior(trace, var_names=['logκ difference of means','logκ difference of stds', 'logκ effect size'],
                  ref_val=0,
                  color='#87ceeb');

In [ ]:
pm.plot_posterior(trace, var_names=['logs difference of means','logs difference of stds', 'logs effect size'],
                  ref_val=0,
                  color='#87ceeb');

## Want something along these lines...

Iterate over true effect size, estimating the effect size from the data.

In [ ]:
es_real = []
es_inferred = {}

for group_κstd in group_κstd_list:
    
    # make true participant level dataset of parameter values
    participant_level_dataset, true_κ_effect_size = make_participant_level_dataset(group_κstd)
    
    # simulate and entire experiment
    trial_level_dataset = simulate_entire_experiment(participant_level_dataset)
    
    # conduct Bayesian inference on parameters
    # This will be a row for each participant, columns include inferred
    inferred_effect_size = bayesian_effect_size_from_discounting_data(trial_level_dataset)
    
    # Store true effect size
    es_real.append(true_κ_effect_size)
    
    # Store posterior median and 95% CI of effect sizes
    es_inferred['point_estimate'] = es_inferred['point_estimate'].append(inferred_effect_size['point_estimate'])
    es_inferred['2.5'] = es_inferred['2.5'].append(inferred_effect_size['2.5'])
    es_inferred['97.5'] = es_inferred['97.5'].append(inferred_effect_size['97.5'])

In [ ]:
# PLOT RESULTS HERE

# References
- Kruschke, John. (2012) Bayesian estimation supersedes the t test. Journal of Experimental Psychology: General.